# Facemask Detection Model

## Table of Contents:

1. [Import Necessary Packages](#import)
2. [Preprocess Dataset](#preprocess)
3. [Decode Images to Numpy Arrays](#decode)
4. [Model Training and Validation](#training)
5. [User Camera Input](#camera)

### __Introduction:__
### This notebook has been created to demonstrate the backend Python code for the Facemask Detection project. Please make sure all data are downloaded to local machine before running this notebook.  
### Follow us on Instagram: https://www.instagram.com/kati.maskdetect/ 
### Our project website: https://kati-test.herokuapp.com/home

## 1. Import Necessary Packages <a name="import"></a>

- pandas framework for reading csv file 
- numpy for storing decoded images
- tensorflow for CNN training
- train_test_split for train validation split
- cv2 for enabling camera

In [1]:
# please make sure all packages are installed on local machine
# if not, use pip install
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import cv2

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


## 2. Preprocess Dataset <a name="preprocess"></a>

In [2]:
# read the csv label file from local disk
labels = pd.read_csv('data/train_labels_cleaned.csv')

# select binary labels only and rename them
binary_labels = labels[['filename', 'label (0/1)']]
label_only = labels[['label (0/1)']]
binary_labels = binary_labels.rename(columns={'label (0/1)': 'label'})

# concatenate the path name with the file name
data_dir = 'data/images/'
filepath = [[filename, data_dir + filename] for filename in labels['filename']]

# save the paths to a pandas dataframe
pd_filepath = pd.DataFrame(filepath, columns=['filename', 'filepath'])

# this data table contains file name, file path, and labels
data = pd.merge(pd_filepath, binary_labels, how='inner', on='filename')

print(data.head())

      filename                 filepath  label
0  Image_1.jpg  data/images/Image_1.jpg      0
1  Image_2.jpg  data/images/Image_2.jpg      0
2  Image_3.jpg  data/images/Image_3.jpg      0
3  Image_4.jpg  data/images/Image_4.jpg      0
4  Image_5.jpg  data/images/Image_5.jpg      0


## 3. Decode Images to Numpy Arrays <a name="decode"></a>

In [3]:
# convert an image file to an array
def decode_img(img):
    # https://www.tensorflow.org/api_docs/python/tf/io/decode_jpeg
    # please see the above link for the following function: tf.image.decode_jpeg
    img = tf.image.decode_jpeg(img, channels=3)
    # uniform picture sizes, this parameter can be tuned
    return tf.image.resize(img, [100, 100])


# x is the array of images
x = []
# y is the array of labels
y = []

# iterate through the data table
for index, row in data.iterrows():
    # read image according the filepath
    img = tf.io.read_file(row['filepath'])
    # decode, than convert to numpy format
    img = decode_img(img).numpy()
    # read label
    label = row['label']
    # append them to the arrays
    x.append(img)
    y.append(label)

# convert x, y to numpy arrays
y = np.array(y)
x = np.array(x)

## 4. Model Training and Validation <a name="training"></a>

In [4]:
# train, validation split
# no test data since we will use real images as our test data
# random_state=42, 42 is the golden standard for random seed
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# model, the layers in this part can be tuned
# to improve the model, feel free to adjust/add/delete the following layers
model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1. / 255),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# binary crossentropy is a loss function that is used in binary classification tasks
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# epochs can be tuned
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=3)
model.save("model_save")

Epoch 1/3
282/282 [==============================] - 32s 111ms/step - loss: 0.2660 - accuracy: 0.8902 - val_loss: 0.1433 - val_accuracy: 0.9494
Epoch 2/3
282/282 [==============================] - 33s 116ms/step - loss: 0.1135 - accuracy: 0.9597 - val_loss: 0.0963 - val_accuracy: 0.9689
Epoch 3/3
282/282 [==============================] - 33s 118ms/step - loss: 0.0788 - accuracy: 0.9726 - val_loss: 0.0971 - val_accuracy: 0.9694
INFO:tensorflow:Assets written to: model_save\assets


## 5. User Camera Input <a name="camera"></a>

In [11]:
# create a video capture
cap = cv2.VideoCapture(0)


def camera_input_color(cap):
    status = ""
    while True:
        # capture frame
        ret, frame = cap.read()

        font = cv2.FONT_HERSHEY_SIMPLEX

        # use putText() method for inserting text on video
        cv2.putText(frame, status, (50, 50), font, 1, (0, 255, 255), 2, cv2.LINE_4)

        color = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
        # display the frame
        cv2.imshow('frame', color)
        # save the image continuously on local disk
        cv2.imwrite('data/captured_images/c1.jpg', frame)
        # read the image
        captured_img = tf.io.read_file('data/captured_images/c1.jpg')
        # decode the image and store it in a numpy array
        captured_img = decode_img(captured_img).numpy()
        # create a batch
        captured_img = (np.expand_dims(captured_img, 0))
        # predict the image
        prob = model.predict(captured_img)
        # 0.9 can be tuned
        if prob[0][0] > 0.9:
            status = "With mask"
        else:
            status = "No mask"

        # turn off the camera on signal "q"
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


def release_camera(cap):
    # release the capture
    cap.release()
    cv2.destroyAllWindows()


camera_input_color(cap)
release_camera(cap)